In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip install -q bitsandbytes accelerate transformers datasets peft sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
assert torch.cuda.is_available(), "GPU is required"
print("GPU:", torch.cuda.get_device_name(0))

GPU: NVIDIA A100-SXM4-40GB


In [ ]:
from transformers import (
    LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import pandas as pd

MODEL_ID = "meta-llama/Llama-3.1-8B"
MAX_LENGTH = 512
BNB_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

PEFT_CONFIG = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj"]
)


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP/Datasets/lyrics_song_info.csv")

if 'primary_genre' not in df.columns and 'tags' in df.columns:
    df['primary_genre'] = (
        df['tags']
        .fillna('')
        .str.split(';')
        .map(lambda L: L[0] if L else None)
    )
df = df.dropna(subset=['lyrics','primary_genre'])

ds_genre = Dataset.from_pandas(df[['lyrics','primary_genre']])


In [ ]:
!pip install -q huggingface_hub

In [ ]:
from transformers import LlamaTokenizer

MODEL_ID = "meta-llama/Llama-3.1-8B"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    token=True,
)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def preprocess_genre(examples):
    texts = [
        f"Lyrics: {ly}\nGenre: {gn}"
        for ly, gn in zip(examples['lyrics'], examples['primary_genre'])
    ]
    tok = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
    )
    tok["labels"] = tok["input_ids"].copy()
    return tok

tok_genre = ds_genre.map(
    preprocess_genre,
    batched=True,
    remove_columns=ds_genre.column_names
)

Map:   0%|          | 0/37907 [00:00<?, ? examples/s]

In [ ]:
model = LlamaForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=BNB_CONFIG,
    device_map="auto",
)
model = get_peft_model(model, PEFT_CONFIG)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer, mlm=False
)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/llama/llama3-genre-qlora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=3e-4,
    logging_steps=50,
    bf16=True,
    save_total_limit=1,
)

from transformers import default_data_collator

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_genre,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)


trainer.train()
trainer.save_model("/content/drive/MyDrive/llama/llama3-genre-qlora")

<ipython-input-18-bd16e814c358>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,2.281500
100,2.285500
150,2.242500
200,2.226600
250,2.256700
300,2.207900
350,2.230100
400,2.187600
450,2.255700
500,2.237700


In [ ]:
def classify_lyrics(lyrics: str, max_new_tokens: int = 10) -> str:
    prompt = f"Lyrics: {lyrics}\nGenre:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=MAX_LENGTH
    ).to(device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )
        gen = tokenizer.decode(
        out[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )
    return gen.strip().split("\n")[0]

print(classify_lyrics("Hey, I know you want to dance with me on the floor Just put your hands up girl and give me some more There's no time to spare, so give me what you've got So rock that body girl so hard till it drops", 10))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Rap R&B Pop Dance
